In [8]:
import os, time
from os.path import exists, join

import cv2
import glob
import numpy as np

from matplotlib import pyplot as plt
from IPython.display import clear_output

In [2]:
def try_find_path( p ):
    if not exists( p ):
        raise Exception( "%s is not found." % p )

In [3]:
def read_grayscale( p ):
    return cv2.imread( p, cv2.IMREAD_GRAYSCALE )

In [4]:
def show_img( origin_img, tissue_type, old_img, new_img ):
    fig, (a1, a2) = plt.subplots( 1,  2, figsize=(25,25) )
    a1.imshow(origin_img)
    a1.set_title(tissue_type)
    a2.imshow(new_img, cmap='gray')
    a2.set_title('new')
#     a2.imshow(old_img, cmap='gray')
#     a2.set_title('old')
#     a3.imshow(new_img, cmap='gray')
#     a3.set_title('new')
    fig.show()
    
def show_images(images):
    width, height = 25, 25
    fig           = fig=plt.figure(figsize=(width, height))
    columns       = 2
    rows          = round((len(images) / columns) + 0.5 )
    print("row", rows)
    for i in range(len(images)):
        img_path = images[i]
        img = cv2.imread( img_path, cv2.IMREAD_COLOR )
        img = cv2.cvtColor( img , cv2.COLOR_BGR2RGB )
        f = fig.add_subplot(rows, columns, i+1)
        f.title.set_text('{}:    {}'.format(i, img_path.split(os.sep)[-1]))
        plt.imshow(img)
    plt.show()


In [18]:
def checkDoNothing(image_paths):
    try:
        if not len(image_paths) > 0 or not type(image_paths) == list: return False
        
        img = read_grayscale( image_paths[0] )
        tmp = np.zeros_like(img)
        for image_path in image_paths:
            if '.jpg' in image_path:
                continue
            img = read_grayscale(image_path) 
            img[img > 0] = 1
            
#             uniq, count_uniq = np.unique(img, return_counts=True)
#             print( image_path, '\n', uniq, count_uniq)
            
            tmp += img
        tmp[tmp == 1] = 0
#         uniq, count_uniq = np.unique(tmp, return_counts=True)
#         print( uniq, count_uniq)
        count_no_zero = np.count_nonzero(tmp)
        return True if count_no_zero == 0 else False 
    except Exceptions as e:
        print('error check dup', e)
        return False 
        

In [19]:
def process( selected_dir, dir_name ):
#     Images_Medetec_dir = join( os.getcwd(), dataset_name)
#     try_find_path(Images_Medetec_dir)
    
#     dir_name = input( "dir: " )
#     selected_dir = join( Images_Medetec_dir, dir_name )
    try_find_path( selected_dir )
    
    all_images = glob.glob(selected_dir + "/*.jpg") + glob.glob(selected_dir + "/*.png")
    versions = [ x.split(os.sep)[-1].split('_')[-1].split('.')[0][-3:] for x in all_images if '.png' in x]
    last_version = sorted(set(versions))[-1]
    old_version = [x for x in set(versions) if x != last_version ]
    
    
#     print("last_version: ", last_version)
    images = [x for x in all_images if '.jpg' in x or "{}.png".format(last_version) in x]

    images = sorted(images)
    do_noting = checkDoNothing(images)
    
    print(last_version," -> ", do_noting, " :\t", selected_dir)
    
    if do_noting: 
        return
    
    remove_olds = []
    for version in old_version:
        for file_type in ['tif', 'png']: 
            remove_olds += glob.glob(selected_dir + "/*{}.{}".format(version, file_type))
   
    print("remove_images", remove_olds)
    for i in remove_olds:
        os.remove(i)
        
    show_images(images)
    
    original_img_name = "%s.jpg" % dir_name
    original_img_path = join( selected_dir , original_img_name )
    try_find_path( original_img_path )

    original_img = cv2.cvtColor( cv2.imread( original_img_path, cv2.IMREAD_COLOR ) , cv2.COLOR_BGR2RGB )
    overlay = original_img.copy() 
    output = original_img.copy()
    
    alpha = 0.2
    
    step_list = input( "step list: " )
    step_list = [int(x) for x in step_list]
    print(type(step_list), step_list)
    
    if 'n' in step_list:
        return 
    
    result_imgs = []
    merge_tissue = np.zeros((original_img.shape[0], original_img.shape[1]))     
    
    for index, value in enumerate(step_list):
        
        sub_overlay = original_img.copy() 
        sub_output = original_img.copy()
        tissue_type = images[value].split('os.sep')[-1]
    

        
        print('step ', value, tissue_type)
        background_img_path = images[value]
        background_img = read_grayscale( background_img_path )
        new_img = background_img.copy()
        for restValue in step_list[index + 1:]:
            print(restValue)
            foreground_img = read_grayscale( images[restValue] )
            new_img[foreground_img != 0 ] = 0
        
        result_imgs += [[background_img_path, new_img]]    
        print(tissue_type, 'empty' in tissue_type)
        if not 'empty' in tissue_type:
            merge_tissue[new_img > 0] = 1
            
            sub_overlay[new_img > 0] = (98, 255, 46)
            cv2.addWeighted(sub_overlay, alpha, sub_output, 1 - alpha, 0, sub_output)
            show_img(sub_output, tissue_type, background_img, new_img)
            plt.show()
        
    overlay[merge_tissue > 0] = (98, 255, 46)
    cv2.addWeighted(overlay, alpha, output, 1 - alpha, 0, output)
    show_img(output, 'merge tissue', merge_tissue, merge_tissue)
    plt.show()
    
    override = input("Overwrite? (Y/N):") in ["Y", "y"]

    if override:
        for [path, img] in result_imgs:
            cv2.imwrite( path, img )
    else:
        pass        
#         new_name = input("new name: ") + ".png"
#         new_path = join( selected_dir, new_name )
#         cv2.imwrite( new_path, new_img )


# Loop checker

In [23]:
tmp_dir = join( os.getcwd(), 'set5')
start_time = time.process_time()
print('start process: %s\n' %start_time)
for root, dirs, files in os.walk(tmp_dir):
    if(root == tmp_dir):
        for sub_dir in dirs:
            if not sub_dir.isdecimal():
                continue
            full_sub_dir = join(root, sub_dir)
            process(full_sub_dir, sub_dir)
            print("sub process end at: {}\n".format(time.process_time() - start_time))

print("END", time.process_time() - start_time)

start process: 57.09375

001  ->  True  :	 G:\New folder\ton_set1\set5\100361
sub process end at: 0.109375

001  ->  True  :	 G:\New folder\ton_set1\set5\100362
sub process end at: 0.21875

001  ->  True  :	 G:\New folder\ton_set1\set5\100363
sub process end at: 0.3125

001  ->  True  :	 G:\New folder\ton_set1\set5\100365
sub process end at: 0.421875

001  ->  True  :	 G:\New folder\ton_set1\set5\100366
sub process end at: 0.53125

001  ->  True  :	 G:\New folder\ton_set1\set5\100367
sub process end at: 0.640625

001  ->  True  :	 G:\New folder\ton_set1\set5\100368
sub process end at: 0.734375

001  ->  True  :	 G:\New folder\ton_set1\set5\100369
sub process end at: 0.84375

001  ->  True  :	 G:\New folder\ton_set1\set5\100370
sub process end at: 0.953125

001  ->  True  :	 G:\New folder\ton_set1\set5\100371
sub process end at: 1.0625

001  ->  True  :	 G:\New folder\ton_set1\set5\100373
sub process end at: 1.1875

001  ->  True  :	 G:\New folder\ton_set1\set5\100374
sub process end at

# Remove empty image

In [13]:
def clearEmptySpaceImage(image_dir):
    try:
        emptry_paths = glob.glob(join(image_dir, '*empty*'))
        for empty_space_path in emptry_paths:
            os.remove(empty_space_path)
        return True, [x.split(os.sep)[-2] for x in emptry_paths]
    except Exception as e:
        print('error', e)
        return False, []

try_find_path(tmp_dir)
print(tmp_dir)

for root, dirs, files in os.walk(tmp_dir):
    if(root == tmp_dir):
        for sub_dir in dirs:
            if not sub_dir.isdecimal():
                continue
            full_sub_dir = join(root, sub_dir)
            status, files = clearEmptySpaceImage(full_sub_dir)
            if status and len(files) > 0:
                print("{}\t{}".format(sub_dir, ", ".join(files)))

G:\New folder\ton_set1\set5


In [ ]:
root         = os.getcwd() + "/data"
dataset_name = "wound_rajavithi"
image_dir    = join( root, dataset_name, "wound_classification", "image_multimarker" )
print(image_dir)

In [ ]:

count = 0

for path in os.walk(image_dir):
    path_name = None
    
    if path[0] != image_dir:
        count    += 1
        path_name = path[0]
        images    = []
    
        print(path_name)
        for image in glob.glob(path_name + "/*.jpg"):
            images.append(image)
        for image in glob.glob(path_name + "/*.png"):
            images.append(image)
            
        images = sorted(images)
        show_images(images)
        
        
        
    if count == 1:
        break

In [ ]:
IMG_2375
IMG_2477
IMG_2567
IMG_2677
IMG_2678
IMG_2775
IMG_2991
IMG_2992
IMG_3045
IMG_3086
IMG_3121
IMG_3125
IMG_3126
IMG_3137
IMG_3138
IMG_3139
IMG_3140
IMG_3153
IMG_3235
IMG_3236
IMG_3318
IMG_3321
IMG_3322
IMG_3323
IMG_3325
IMG_3353
IMG_3379
IMG_3381
IMG_3383
IMG_3385
IMG_3386
IMG_3387
IMG_3441
IMG_3442
IMG_3443
IMG_3460
IMG_3461
IMG_3609
IMG_4406
IMG_4407
IMG_4814
IMG_4888
IMG_4893
IMG_4894
IMG_4895
IMG_4904
IMG_4905
IMG_4907
IMG_4908
IMG_5426
IMG_5486
IMG_5487
IMG_6032
IMG_6033
IMG_6080
IMG_8054
IMG_8568
IMG_8580
IMG_8582
IMG_8961
IMG_9473
IMG_9474
IMG_9475
IMG_9476
IMG_9479
IMG_9481
IMG_9483
IMG_9484
IMG_9485
IMG_25570204_081244
IMG_25570207_094611
IMG_25570207_100736
IMG_25570207_100924
IMG_25570207_101223
IMG_25570207_101229
IMG_25570207_101233
IMG_25570207_101736
IMG_25570207_104428
IMG_25570207_104758
IMG_25570207_104804
IMG_25570207_112010
IMG_25570207_112013
IMG_25570207_113044
IMG_25570221_095738
IMG_25570221_095741